In [ ]:
# reclassification: 1-9 = low-high
import rasterio
import numpy as np

input_file = "GHI_B.tif"
output_file = "GHI_RC.tif"

def reclassify_raster(input_file, output_file, num_classes=9):
    """
    Reclassify a raster into equal intervals with values ranging from 1 to num_classes.
    
    :param input_file: Path to input raster
    :param output_file: Path to output reclassified raster
    :param num_classes: Number of classes (default: 9)
    """
    with rasterio.open(input_file) as src:
        profile = src.meta.copy()
        profile.update(dtype=rasterio.uint8, nodata=None)  # No NoData value

        band_data = src.read(1)

        # Generate equal intervals
        intervals = np.linspace(band_data.min(), band_data.max(), num_classes + 1)

        reclassified_data = np.digitize(band_data, bins=intervals, right=True).astype(np.uint8)

        with rasterio.open(output_file, "w", **profile) as dst:
            dst.write(reclassified_data, 1)

    print(f"Reclassified raster saved to {output_file}")

reclassify_raster(input_file, output_file)

In [ ]:
# reclassification: 1-9 = high-low
import rasterio
import numpy as np

input_file = "Dem_B.tif"
output_file = "DEM_RC.tif"

with rasterio.open(input_file) as src:
    data = src.read(1)
    profile = src.profile
    
    # Handle NoData values
    nodata = src.nodata
    if nodata is not None:
        data = np.where(data == nodata, np.nan, data)

min_val, max_val = np.nanmin(data), np.nanmax(data)

reclassified_data = np.full_like(data, fill_value=-9999, dtype=np.int16)

interval = (max_val - min_val) / 9

for i in range(9):
    lower_bound = min_val + i * interval
    upper_bound = min_val + (i + 1) * interval
    reclassified_data[(data > lower_bound) & (data <= upper_bound)] = 9 - i  # Assign values from 9 to 1

reclassified_data[np.isnan(data)] = -9999
reclassified_data[reclassified_data == 0] = -9999  # Prevent 0 values

profile.update(dtype=rasterio.int16, count=1, nodata=-9999)

with rasterio.open(output_file, "w", **profile) as dst:
    dst.write(reclassified_data, 1)

print(f"Reclassification completed. Output saved to: {output_file}")

In [ ]:
# Irregular reclassification
import rasterio
import numpy as np

input_file = "Aspect.tif"
output_file = "Aspect_RC.tif"

# Open the input raster file
with rasterio.open(input_file) as src:
    data = src.read(1)  # Read the first band
    profile = src.profile.copy()  # Copy metadata

# Create an array for storing the reclassified data
reclassified_data = np.zeros_like(data, dtype=np.uint8)

# Apply reclassification rules
reclassified_data[(data == 1) | (data == 5)] = 9  # Values 1 and 5 → 9
reclassified_data[(data == 3) | (data == 7)] = 1  # Values 3 and 7 → 1
reclassified_data[(data != 1) & (data != 5) & (data != 3) & (data != 7)] = 5  # Others → 5

# Update metadata
profile.update(dtype=rasterio.uint8, nodata=None)

# Save the reclassified raster
with rasterio.open(output_file, "w", **profile) as dst:
    dst.write(reclassified_data, 1)

print(f"Reclassification completed. Output saved to: {output_file}")


In [ ]:
# Irregular reclassification 2
import rasterio
import numpy as np

input_file = "Slope_Angle.tif"
output_file = "Slope_RC.tif"

def reclassify_raster(input_file, output_file):
    """
    Reclassify a raster into 9 classes (from 9 to 1).
    - Values above 11.31 are set to 0.
    """
    with rasterio.open(input_file) as src:
        data = src.read(1)
        profile = src.profile.copy()

        # Set values above 11.31 to 0
        data = np.where(data > 11.31, 0, data)

        valid_data = data[data > 0]

        reclassified_data = np.zeros_like(data, dtype=np.uint8)

        if valid_data.size > 0:
            min_val, max_val = valid_data.min(), valid_data.max()
            interval = (max_val - min_val) / 9

            for i in range(9):
                lower_bound = min_val + i * interval
                upper_bound = min_val + (i + 1) * interval
                reclassified_data[(data > lower_bound) & (data <= upper_bound)] = 9 - i

        reclassified_data[data == 0] = 0

        profile.update(dtype=rasterio.uint8, count=1, nodata=0)

        with rasterio.open(output_file, "w", **profile) as dst:
            dst.write(reclassified_data, 1)

    print(f"Reclassification completed. Output saved to: {output_file}")

reclassify_raster(input_file, output_file)

In [ ]:
# weighted sum
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import reproject
import numpy as np

raster_paths = [
    "GHI_RC.tif", "Temperature_RC.tif", "Aspect_RC.tif",
    "Slope_RC.tif", "Grid_RC.tif", "Humidity_RC.tif",
    "Road_RC.tif", "DEM_RC.tif", "Urban_RC.tif"
]

weights = [0.3781, 0.1533, 0.1118, 0.1061, 0.1058, 0.0622, 0.0354, 0.0314, 0.0158]

default_crs = "EPSG:4326"

with rasterio.open(raster_paths[0]) as template:
    template_profile = template.profile
    template_transform = template.transform
    template_width, template_height = template.width, template.height
    template_crs = template.crs or default_crs

    print(f"Template Info: CRS={template_crs}, Size={template_width}x{template_height}")

weighted_sum = np.zeros((template_height, template_width), dtype=np.float32)

for raster_path, weight in zip(raster_paths, weights):
    print(f"Processing: {raster_path}, Weight: {weight}")
    try:
        with rasterio.open(raster_path) as src:
            src_crs = src.crs or default_crs
     
            if src_crs != template_crs or src.width != template_width or src.height != template_height:
                print("  Resampling/Reprojecting...")
                reprojected_data = np.zeros((template_height, template_width), dtype=np.float32)

                reproject(
                    source=rasterio.band(src, 1),
                    destination=reprojected_data,
                    src_transform=src.transform,
                    src_crs=src_crs,
                    dst_transform=template_transform,
                    dst_crs=template_crs,
                    resampling=Resampling.bilinear,
                    num_threads=4
                )
            else:
                print("  Matching template, using original data.")
                reprojected_data = src.read(1)

            weighted_sum += reprojected_data * weight

    except Exception as e:
        print(f"Error processing {raster_path}: {e}")
        continue

output_path = "Weighted_sum.tif"
template_profile.update(dtype=rasterio.float32, crs=template_crs)

try:
    with rasterio.open(output_path, "w", **template_profile) as dst:
        dst.write(weighted_sum, 1)
    print(f"Weighted sum saved to: {output_path}")
except Exception as e:
    print(f"Error saving result: {e}")

In [ ]:
# subtract constraint layers
import rasterio
from rasterio.warp import reproject, Resampling
import numpy as np

weighted_sum_path = "Weighted_sum.tif"
proad_path = "PRoad.tif"
urban_path = "Urban.tif"
grid_path = "Grid.tif"
protected_area_path = "Protected_area.tif"
cropland_area_path = "Cropland.tif"
water_area_path = "Water.tif"
output_path = "Final_sum.tif"

with rasterio.open(weighted_sum_path) as weighted_sum:
    weighted_sum_data = weighted_sum.read(1)
    profile = weighted_sum.profile
    weighted_transform = weighted_sum.transform
    weighted_crs = weighted_sum.crs
    weighted_shape = weighted_sum_data.shape

def resample_to_match(source_path, target_transform, target_crs, target_shape):
    with rasterio.open(source_path) as src:
        src_data = src.read(1)
        src_transform = src.transform
        src_crs = src.crs or target_crs

        destination = np.zeros(target_shape, dtype=np.float32)

        reproject(
            source=src_data,
            destination=destination,
            src_transform=src_transform,
            src_crs=src_crs,
            dst_transform=target_transform,
            dst_crs=target_crs,
            resampling=Resampling.nearest
        )
    return destination

final_mask = np.zeros_like(weighted_sum_data, dtype=bool)

for mask_path in [proad_path, urban_path, grid_path, protected_area_path, cropland_area_path, water_area_path]:
    print(f"Processing: {mask_path}")
    mask_resampled = resample_to_match(mask_path, weighted_transform, weighted_crs, weighted_shape)

    final_mask = np.logical_or(final_mask, mask_resampled == 1)

final_class = np.where(final_mask, np.nan, weighted_sum_data)

profile.update(dtype=rasterio.float32, nodata=np.nan)

with rasterio.open(output_path, "w", **profile) as dst:
    dst.write(final_class, 1)

print(f"Final_sum.tif created successfully at: {output_path}")

In [ ]:
# final reclassification
import numpy as np
import rasterio

input_path = "Final_sum.tif"
output_path = "Final_sum_reclass.tif"

with rasterio.open(input_path) as src:
    data = src.read(1)
    profile = src.profile
    nodata_value = src.nodata

classified_data = np.zeros_like(data, dtype=np.float32)

classified_data[(data > 1) & (data <= 3)] = 4
classified_data[(data > 3) & (data <= 5)] = 3
classified_data[(data > 5) & (data <= 7)] = 2
classified_data[(data > 7) & (data <= 9)] = 1

if nodata_value is not None:
    classified_data[data == nodata_value] = np.nan

profile.update(dtype=rasterio.float32, nodata=np.nan)

with rasterio.open(output_path, "w", **profile) as dst:
    dst.write(classified_data, 1)

print(f"Reclassification completed. Output saved to: {output_path}")